# Chapter 3 Slides
## Ethem Alpaydin - Introduction to Machine Learning

Open source notebooks/slides for Ethem Alpaydin's Introduction to Machine Learning with examples in Python 3.6.

In [1]:
import numpy as np
from support import basic_plot, np_map

from bokeh.plotting import Figure, output_notebook, show
from bokeh.models import Range1d, FixedTicker
from bokeh.layouts import gridplot
# from bokeh.palettes import Colorblind3 as colors
colors = ['#f55385', '#519aba', '#8dc149']

from scipy.stats import norm

In [2]:
import sys
sys.path.insert(0, '/home/vagrant/notebooks')
from custom_theme import output_notebook_themed
output_notebook_themed()

# output_notebook()

Loading BokehJS ...

# Bayesian Decision Theory

Probability theory is a framework for making decisions under uncertainty.

In coin tossing, for instance, the only observed variable is the toss.

\\[x = f({\mathbf{z}})\\]

The coin toss can't be modelled deterministically, so a random variable is used instead.

\\[P (X = x)\\]

Let us say \\(X = 1\\) denotes that the outcome of a toss is heads and \\(X = 0\\) denotes tails. Then the outcome is a Bernoulli random variable: 

\\[P(X = 1) = {p_0}{\text{ and }}P(X = 0) = 1 - P(X = 1) = 1 - {p_0}\\]

If \\(p_0 > 0.5\\), then we predict heads will be flipped, because it is the more probably choice. 

We can estimate the value of \\(p_0\\) from a sample.

\\[{\hat p_0}{\text{  =  }}\frac{{{\text{# \{tosses with outcome heads\} }}}}{{{\text{# \{tosses\} }}}}\\]

In [3]:
# Draw a sample of 1000 coin flips:

nobs = 100
p_0 = 0.5
outcomes = np.random.binomial(1, p_0, nobs)

print(outcomes)

[0 1 0 0 0 1 0 0 1 0 1 1 0 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 1 0
 0 1 1 1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 1
 0 0 1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 0 0 1 1 1 0 0 1 0]


In [4]:
def bernoulli_trials(p_0 = 0.5, nobs=100, show_action=False):
    outcomes = np.random.binomial(1, p_0, nobs)

    plot = basic_plot(width=550, height=200, title='Coin Flips, p̂ = ' + str(outcomes.sum()/len(outcomes)))
    plot.scatter(x=range(len(outcomes)), y=list(outcomes), size=5, alpha=1,
                 line_color=None, color=colors[0])
    plot.yaxis.ticker = FixedTicker(ticks=[0, 1])

    plot_binary = basic_plot(width=50, height=200)
    plot_binary.vbar(x=[0,1], top=[outcomes.sum(), len(outcomes)-outcomes.sum()], width=0.75, color=colors[1])
    plot_binary.xaxis.ticker = FixedTicker(ticks=[0, 1])
    plot_binary.y_range = Range1d(0,(max([outcomes.sum(), len(outcomes)-outcomes.sum()])+5))

    if show_action:
        plot.title.text = 'Coin Flips, p̂ = ' + str(outcomes.sum()/len(outcomes)) + ' predict: class = ' + \
                        str(np.argmax(np.array([outcomes.sum()/len(outcomes), 1-outcomes.sum()/len(outcomes)])))
    
    return(gridplot([[plot, plot_binary]], 
                    toolbar_location=None))

In [5]:
show(bernoulli_trials(p_0=0.5))

# Losses and Risk


If the risk of taking all actions is equal, then the loss function is:

\\[{\lambda _{ik}} = \left\{ \begin{gathered}
  0{\text{ if }}i = k \hfill \\
  1{\text{ if }}i \ne k \hfill \\ 
\end{gathered}  \right.\\]



And the risk of an action can be calculated as:

\\[\begin{gathered}
  R({\alpha _i}|{\mathbf{x}}) = \sum\limits_{k = 1}^K {{\lambda _{ik}}P({C_k}|{\mathbf{x}})}  \hfill \\
   = \sum\limits_{k = 1}^K {P({C_k}|{\mathbf{x}})}  \hfill \\
   = 1 - P({C_i}|{\mathbf{x}}) \hfill \\ 
\end{gathered} \\]

For minimum risk, choose the most probable class.

In [6]:
show(bernoulli_trials(p_0=0.3, show_action=True))

In [7]:
show(bernoulli_trials(p_0=0.8, show_action=True))

# Classification

Say for a data sample our priors are defiend by a MLE of the Bernoulli distribution for each class of examples, with the parameter P:

\begin{equation}
\hat P = \frac{{\sum {{x^t}} }}{N}
\end{equation}

Our liklihoods are defiend by a MLE of the Normal distribution, with the parameters mean and standard deviation:

\begin{equation}
p(x|\mu ,{\sigma}) = \frac{1}{{\sqrt {2{\sigma ^2}\pi } }}\exp \left[ {\frac{{{{(x - \mu )}^2}}}{{2{\sigma ^2}}}} \right], - \infty  < x < \infty
\end{equation}

We use the univariate normal density function provided in SciPy to calulate values or vectors:

In [8]:
# norm.pdf(INPUT_SAMPLE_OR_VECTOR, loc=MEAN_VALUE, 
#          scale=STANDARD_DEVIATION_VALUE)

We can use Bayes theorem to find a posterior probability for a single sample \\(x^t\\):
\\[p({C_1|\mathbf{x}}) = \frac{p({\mathbf{x}}|C_1)\hat P(C_1)}{p({\mathbf{x}}|C_1)\hat P(C_1) + p({\mathbf{x}}|C_2)\hat P(C_2)}\\]

In [9]:
prior_1 = 0.3
prior_2 = 1 - prior_1
liklihood_1_mean = -1.5
liklihood_1_std = 0.5
liklihood_2_mean = 1
liklihood_2_std = 0.8

x_t = -0.4

# normal density function: loc = mean, scale = standard deviation
print("P(C_1|x): ", 
       norm.pdf(x_t, loc=liklihood_1_mean, scale=liklihood_1_std)*prior_1 /  \
      (norm.pdf(x_t, loc=liklihood_1_mean, scale=liklihood_1_std)*prior_1+  \
       norm.pdf(x_t, loc=liklihood_2_mean, scale=liklihood_2_std)*prior_2))
print("P(C_2|x): ", 
       norm.pdf(x_t, loc=liklihood_2_mean, scale=liklihood_2_std)*prior_2 /  \
      (norm.pdf(x_t, loc=liklihood_1_mean, scale=liklihood_1_std)*prior_1+  \
       norm.pdf(x_t, loc=liklihood_2_mean, scale=liklihood_2_std)*prior_2))

P(C_1|x):  0.21993516645994293
P(C_2|x):  0.7800648335400571


In [10]:
# generate a set of uniformly distributed points for plotting
x_min = min([norm.ppf(.00001, loc=liklihood_1_mean, scale=liklihood_1_std),
     norm.ppf(.00001, loc=liklihood_2_mean, scale=liklihood_2_std)])
x_max = max([norm.ppf(0.99999, loc=liklihood_1_mean, scale=liklihood_1_std),
     norm.ppf(0.99999, loc=liklihood_2_mean, scale=liklihood_2_std)])
x = np.linspace(x_min, x_max, 200)

# calculate over a range for plotting
liklihood_1 = norm.pdf(x, loc=liklihood_1_mean, scale=liklihood_1_std)
liklihood_2 = norm.pdf(x, loc=liklihood_2_mean, scale=liklihood_2_std)

evidence = liklihood_1*prior_1+liklihood_2*prior_2

plot_liklihoods = basic_plot(title='Liklihoods')
plot_liklihoods.line(x=x, y=liklihood_1, legend='p(x|C₁)', color=colors[0], line_width=3)
plot_liklihoods.line(x=x, y=liklihood_2, legend='p(x|C₂)', color=colors[1], line_width=3)
plot_liklihoods.x_range = Range1d(x_min, x_max)
plot_liklihoods.y_range = Range1d(0, 1)

In [11]:
show(plot_liklihoods)

In [12]:
plot_liklihoods_priors = basic_plot(title='Liklihoods * Priors')
plot_liklihoods_priors.line(x=x, y=liklihood_1*prior_1, legend='p(x|C₁)*P(C₁)', color=colors[0], line_width=3)
plot_liklihoods_priors.line(x=x, y=liklihood_2*prior_2, legend='p(x|C₂)*P(C₂)', color=colors[1], line_width=3)
plot_liklihoods_priors.x_range = Range1d(x_min, x_max)
plot_liklihoods_priors.y_range = Range1d(0, 1)

In [13]:
show(plot_liklihoods_priors)

In [14]:
def make_posterior_plot():
    plot_posterior = basic_plot(title='Posterior = Liklihoods * Priors / Evidence')
    plot_posterior.line(x=x, y=(liklihood_1*prior_1)/evidence, legend='p(x|C₁)*P(C₁)/P(X)', color=colors[0], line_width=3)
    plot_posterior.line(x=x, y=(liklihood_2*prior_2)/evidence, legend='p(x|C₂)*P(C₂)/P(X)', color=colors[1], line_width=3)
    plot_posterior.x_range = Range1d(x_min, x_max)
    plot_posterior.y_range = Range1d(0, 1)
    return(plot_posterior)

In [15]:
show(make_posterior_plot())

Again, in the case of equal losses, we choose the more probable class by interpreting the posterior probability.

In [16]:
def prob_class_1(x_t):
    prob = norm.pdf(x_t, loc=liklihood_1_mean, scale=liklihood_1_std)*prior_1 /  \
          (norm.pdf(x_t, loc=liklihood_1_mean, scale=liklihood_1_std)*prior_1+  \
           norm.pdf(x_t, loc=liklihood_2_mean, scale=liklihood_2_std)*prior_2)
    return prob

def make_posterior_plot_with_loss(PC_1=0.5):
    x_post = np.linspace(x_min, x_max, 100)
    post_class1 = (prob_class_1(x_post) > PC_1)
    color_post = np_map({0: '#1d5b91', 1: '#63213f'}, post_class1)

    plot_posterior = make_posterior_plot()
    plot_posterior.rect(x=x_post, y=0.5, height=1, width=(x_max-x_min)/100, color=color_post, alpha=1, level='underlay')
    return(plot_posterior)

In [17]:
show(make_posterior_plot_with_loss())

# Unequal Losses

<p>
<center>Loss Table ($\lambda_{ik}$)</center>

|                          | Actual $C_1$  | Actual $C_2$ |
| ------------------------ |:-------------:| :-----------:|
| $\alpha_1$: choose $C_1$ | 0             | 10           |
| $\alpha_2$: choose $C_2$ | 5             | 0            |

\\[\begin{gathered}
  R({\alpha _1}|{\mathbf{x}}) = 0 \cdot P({C_1}|{\mathbf{x}}) + 10 \cdot P({C_2}|{\mathbf{x}}) \hfill \\
  R({\alpha _1}|{\mathbf{x}}) = 10 \cdot (1 - P({C_1}|{\mathbf{x}})) \hfill \\
   \hfill \\
  R({\alpha _2}|{\mathbf{x}}) = 5 \cdot P({C_1}|{\mathbf{x}}) + 0 \cdot P({C_2}|{\mathbf{x}}) \hfill \\
  R({\alpha _2}|{\mathbf{x}}) = 5 \cdot P({C_1}|{\mathbf{x}}) \hfill \\ 
\end{gathered} \\]

\\[\begin{gathered}
  R({\alpha _1}|{\mathbf{x}}) = 10 \cdot (1 - P({C_1}|{\mathbf{x}})) \hfill \\
  R({\alpha _2}|{\mathbf{x}}) = 5 \cdot P({C_1}|{\mathbf{x}}) \hfill \\
   \hfill \\
  {\text{Choose }}{\alpha _1}{\text{ if:}} \hfill \\
  R({\alpha _1}|{\mathbf{x}}) < R({\alpha _2}|{\mathbf{x}}) \hfill \\
  10 \cdot (1 - P({C_1}|{\mathbf{x}})) < 5 \cdot P({C_1}|{\mathbf{x}}) \hfill \\
  10 - 10 \cdot P < 5 \cdot P \hfill \\
  10 < 15 \cdot P \hfill \\
  \tfrac{2}{3} < P({C_1}|{\mathbf{x}}) \hfill \\
  P({C_1}|{\mathbf{x}}) > \tfrac{2}{3} \hfill \\ 
\end{gathered} \\]

In [18]:
show(make_posterior_plot_with_loss(PC_1=(2/3)))

# Unequal Losses with Reject

<p>
<center>Loss Table ($\lambda_{ik}$)</center>

|                          | Actual $C_1$  | Actual $C_2$ |
| ------------------------ |:-------------:| :-----------:|
| $\alpha_1$: choose $C_1$ | 0             | 10           |
| $\alpha_2$: choose $C_2$ | 5             | 0            |
| $\alpha_r$: reject       | 1             | 1            |

Same:
\\[\begin{gathered}
  R({\alpha _1}|{\mathbf{x}}) = 10 \cdot (1 - P({C_1}|{\mathbf{x}})) \hfill \\
  R({\alpha _2}|{\mathbf{x}}) = 5 \cdot P({C_1}|{\mathbf{x}}) \hfill \\ 
\end{gathered} \\]

New:
\\[R({\alpha _r}|{\mathbf{x}}) = 1\\]

\\[\begin{gathered}
  R({\alpha _1}|{\mathbf{x}}) = 10 \cdot (1 - P({C_1}|{\mathbf{x}})) \hfill \\
  R({\alpha _2}|{\mathbf{x}}) = 5 \cdot P({C_1}|{\mathbf{x}}) \hfill \\
  R({\alpha _r}|{\mathbf{x}}) = 1 \hfill \\ 
\end{gathered} \\]

Choose $\alpha_1$ if:

\\[\begin{gathered}
  R({\alpha _1}|{\mathbf{x}}) < R({\alpha _r}|{\mathbf{x}}) \hfill \\
  10 - 10 \cdot P({C_1}|{\mathbf{x}}) < 1 \hfill \\
  -10 \cdot P <  - 9 \hfill \\
  P({C_1}|{\mathbf{x}}) > \tfrac{9}{{10}} \hfill \\ 
\end{gathered} \\]

Choose $\alpha_2$ if:

\\[\begin{gathered}
  R({\alpha _2}|{\mathbf{x}}) < R({\alpha _r}|{\mathbf{x}}) \hfill \\
  P({C_1}|{\mathbf{x}}) < \tfrac{1}{5} \hfill \\
  P({C_2}|{\mathbf{x}}) > \tfrac{4}{5} \hfill \\ 
\end{gathered} \\]

Reject otherwise.

In [19]:
PC_1=0.9
PC_2=0.9
def make_posterior_plot_with_unqeual_loss(PC_1=0.9, PC_2=0.9):
    x_post_grid = np.linspace(x_min, x_max, 100)
    colors = np.zeros(shape=100, dtype=int)
    post_class1 = prob_class_1(x_post_grid)
    colors[post_class1 > PC_1] = 1
    colors[post_class1 < (1-PC_2)] = 2
    color_post = np_map({0: '#000000', 1: '#63213f', 2: '#1d5b91'}, colors)
    plot_posterior = make_posterior_plot()
    plot_posterior.rect(x=x_post_grid, y=0.5, height=1, width=(x_max-x_min)/100, color=color_post, alpha=1, level='underlay')
    return(plot_posterior)

In [20]:
show(make_posterior_plot_with_unqeual_loss(PC_1=(9/10), PC_2=(4/5)))

# Discriminant Functions

Classification can also be seen as implementing a set of discriminant functions.

\\[{g_i}({\mathbf{x}}) = \mathop {\max }\limits_k {g_k}({\mathbf{x}})\\]

\\[{g_i}({\mathbf{x}}) = \left\{ \begin{gathered}
   -R({\alpha _i}|{\mathbf{x}}) \hfill \\
  P({C_i}|{\mathbf{x}}) \hfill \\
  p({C_i}|{\mathbf{x}})P({C_i}) \hfill \\ 
\end{gathered}  \right.\\]

Divide the feature space into $K$ decision regions, $R_1, ... R_k$ where  \\({R_i} = \{ {\mathbf{x}}|{g_i}({\mathbf{x}}) = \mathop {\max }\limits_k {g_k}({\mathbf{x}})\} \\)

The regions are seperated by decision boundaries, surfaces in the feature space where ties occour among the discriminant functions.